In [1]:
!nvidia-smi

Sat Dec  7 20:16:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           On  |   00000000:AF:00.0 Off |                    0 |
| N/A   41C    P0             27W /  250W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Testing HuggingFace pipeline directly, doesn't work without logging in before starting kernel
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")
pipeline("Hey how are you doing today?")

In [1]:
!pip install langchain==0.0.191
!pip install llama-cpp-python==0.1.66
!pip install sentence-transformers
!pip install huggingface_hub
!pip install auto-gptq==0.2.2

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 10.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.0/405.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 42.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.7 MB/s eta

In [ ]:
DEVICE_TYPE = "cuda" if torch.cuda.is_available() else "cpu"
SHOW_SOURCES = True
logging.info(f"Running on: {DEVICE_TYPE}")
logging.info(f"Display Source Documents set to: {SHOW_SOURCES}")

In [ ]:
# Experimented with GGML LLaMa variant, did not work as per expectations
model_id = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_0.bin"

In [7]:
!pip install huggingface_hub

Defaulting to user installation because normal site-packages is not writeable


In [32]:
# Logging in to HuggingFace (needed for accessing LLaMa weights)

In [1]:
!huggingface-cli login

/bin/bash: line 1: huggingface-cli: command not found


In [7]:
!/home/hice1/tchavan3/.local/bin/huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `NLP` has been saved to /home/hice1/tchavan3/.cache/huggingface/stored_tokens
Your token has been saved to /home/hice1/tchavan3/.cache/huggingface/token
Login successful.
The current active token is: `NLP`


In [8]:
#importing libraries and reading the train dataset
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

import pandas as pd
import numpy as np
import warnings

from tqdm import tqdm

traindf = pd.read_csv('/storage/ice1/6/4/tchavan3/technology_train.csv')

In [9]:
import time

In [10]:
#Using LLaMa 3.2 Instruct with 3 Billion parameters for our experiments

import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:56<00:00, 28.50s/it]


In [31]:
traindf['text'][0]

'There are two traditional views of how Americans can be happiest in retirement. In one, they retrofit their homes with grab bars and wheelchair-accessible bathrooms and "age in place" while maintaining long-time community, church and family ties. In the other, they say farewell to friends and frigid winters and head off to Florida or Sun City for a life of leisurely golf and bridge games.\n\nBut a new study from the Center for Retirement Research at Boston College suggests there is another road that leads to happiness. Older Americans who move are somewhat happier and wealthier than those who stay put, but most of their moves are short-distance ones--some made as part of a retirement plan and some in response to a shock, such as being widowed or divorced, the study found.\n\nDespite their differing circumstances, both groups ended up with at least modest gains in happiness from the move. Those who moved after a shock were a little less unhappy than old folks in comparable circumstance

In [23]:
import random

def select_random_line(paragraph):
    lines = paragraph.splitlines()
    lines = [line for line in lines if line.strip()]
    if lines:
        return random.choice(lines)
    else:
        return None

In [24]:
select_random_line(traindf['text'][0])

'There are two traditional views of how Americans can be happiest in retirement. In one, they retrofit their homes with grab bars and wheelchair-accessible bathrooms and "age in place" while maintaining long-time community, church and family ties. In the other, they say farewell to friends and frigid winters and head off to Florida or Sun City for a life of leisurely golf and bridge games.'

In [30]:
messages = [
        {"role": "system", "content": "You are a sports reporter for ESPN, and you write about the latest sports from the world of sports. All of your content is completely new, and you should not write content you have ever read before."},
        {"role": "user", "content": "Write an imaginary 500 word article on sports, incorporating the following words: "+select_random_line(traindf['text'][0])}
    ]
outputs = pipe(
    messages,
    max_new_tokens=500,
)
op = outputs[0]["generated_text"][-1]['content']
op

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'"Unpacking the Great American Move: How Sports Can Relate to the Nation\'s Migration Patterns"\n\nIn a recent study, it was revealed that nearly a third of American homeowners made at least one move over the past 12 years, with a staggering 7% of them making multiple moves within any two-year span. This staggering data has left many to wonder: what\'s behind the great American move? As we delve into the world of sports, we\'ll explore some unexpected parallels between the nation\'s migration patterns and the world of athletics.\n\nOne striking similarity is the concept of "short-distance" moves, where 60% of all relocations were within a 20-mile radius. This phenomenon is eerily reminiscent of the "short-game" mentality in golf, where a well-placed chip shot or a precise bunker shot can make all the difference in a game. Just as a skilled golfer can make a significant impact on the course without leaving the fairway, many homeowners can find new opportunities and improve their quality

In [8]:
# Running a loop over a thousand samples for paraphrasing due to compute constraints
start = time.time()
ops = []
opssumm = []

for i in range(1000,2000):
    print(f"{i}/2000")

    messages = [
        {"role": "system", "content": "You are a food reporter for TLC, and you write about the latest restaurants and food reviews and culinary events. All of your content is completely new, and you should not write content you have ever read before."},
        {"role": "user", "content": "Write a 500 word food or culinary news article, incorporating the following word: "+select_random_line(traindf['text'][i])},
    ]
    outputs = pipe(
        messages,
        max_new_tokens=500,
    )
    op = outputs[0]["generated_text"][-1]['content']
    ops.append(op)

    messagessumm = [
        {"role": "system", "content": "Summarize the given text in a single sentence under 20 words: "+op},
        {"role": "user", "content": "Summarize the given text in a single sentence under 20 words: "+op},
    ]
    outputssumm = pipe(
        messagessumm,
        max_new_tokens=25,
    )
    opsumm = outputssumm[0]["generated_text"][-1]['content']
    opssumm.append(opsumm)

end = time.time()
print(end - start)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1000/2000


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1001/2000


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

1998/2000


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1999/2000


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


20806.341857671738


In [ ]:
ops

In [10]:
len(opssumm)

1000

In [11]:
dffood = pd.DataFrame({'text': ops, 'summary': opssumm})

In [12]:
dffood.to_csv("food-aug.csv")